In [ ]:
!pip install kaggle
from google.colab import files
#files.upload()
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d willarevalo/chexpert-v10-small

!unzip chexpert-v10-small.zip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
401 - Unauthorized
unzip:  cannot find or open chexpert-v10-small.zip, chexpert-v10-small.zip.zip or chexpert-v10-small.zip.ZIP.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip uninstall tensorflow

Found existing installation: tensorflow 2.8.0+zzzcolab20220506162203
Uninstalling tensorflow-2.8.0+zzzcolab20220506162203:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.7/dist-packages/tensorflow-2.8.0+zzzcolab20220506162203.dist-info/*
    /usr/local/lib/python3.7/dist-packages/tensorflow/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-2.8.0+zzzcolab20220506162203


In [ ]:
!pip uninstall tensorflow-gpu
!pip install tensorflow-gpu==1.15.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 411.5 MB 7.0 kB/s 
     |████████████████████████████████| 503 kB 68.8 MB/s 
     |████████████████████████████████| 50 kB 7.5 MB/s 
     |████████████████████████████████| 3.8 MB 68.9 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=6b4bbe1eb3e21b85121854328e7842b15791c38b7e4a32b4ba7506883b83be51
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting u

In [ ]:
import pandas as pd
import numpy as np
import pickle
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
path = "/content/drive/MyDrive/Colab Notebooks/dataset/CheXpert-v1.0-small/"
# csv파일 레이블로 만들기
train = pd.read_csv(path+'train.csv')
label_cols = train.columns[5:]
train[label_cols] = train[label_cols].fillna(0.0)
train[label_cols] = train[label_cols].replace(-1.0, 1.0)
train["ID"] = train["Path"].apply(lambda x: x.split("patient")[1].split("/")[0])

In [ ]:
import os
os.environ['PYTHONPATH'] += ":/content/drive/MyDrive/Colab Notebooks/Thorax_GAN-master/ProGAN"
import sys 
sys.path.append('/content/drive/MyDrive/Colab Notebooks/Thorax_GAN-master/ProGAN')

In [ ]:
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf
import PIL.Image
import imageio
# import tfutils
import matplotlib.pyplot as plt
import os
import sys

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


network_path = "/content/drive/MyDrive/Colab Notebooks/Thorax_GAN-master/CheXpert_256/GAN_CheXpert/013-pgan-CheXpert-preset-v2-1gpu-fp32-HIST/"
output_data_path = "/content/drive/MyDrive/Colab Notebooks/dataset/fake_data3/"

real_data_path = path

names = ['No Finding', 'Enlarged Cardiomediastinum', 'Cardiomegaly', 'Lung Opacity',
       'Lung Lesion', 'Edema', 'Consolidation', 'Pneumonia', 'Atelectasis',
       'Pneumothorax', 'Pleural Effusion', 'Pleural Other', 'Fracture',
       'Support Devices']

tf.InteractiveSession()

with open(network_path + "network-snapshot-016960.pkl", "rb") as file:
  G, D, Gs = pickle.load(file)

In [ ]:
label_vectors = {}
for cat, n in zip(classes_to_generate, [lesion_n, pleural_n, fracture_n]):
    relevant_labels = real_labels[real_labels[cat] == 1]
    new_labels = relevant_labels.sample(n, replace=True)[names].to_numpy()
    label_vectors[cat] = new_labels

for cat, arr in label_vectors.items():
    print("{}: {}".format(cat, arr.shape))

label_vectors = {}
for cat, n in zip(classes_to_generate, [lesion_n, pleural_n, fracture_n]):
    relevant_labels = real_labels[real_labels[cat] == 1]
    new_labels = relevant_labels.sample(n, replace=True)[names].to_numpy()
    label_vectors[cat] = new_labels

for cat, arr in label_vectors.items():
    print("{}: {}".format(cat, arr.shape))

labels_save = {}
for cat in classes_to_generate:
    labels = label_vectors[cat]
    batch = 1
    used_labels = []
    used_imgname = []
    latents_raw = np.random.RandomState(1000).randn(labels.shape[0], *Gs.input_shapes[0][1:])
    total_num = latents_raw.shape[0]

    print("Generating {}".format(cat))
    sys.stdout.flush()

    for n in range(int(total_num / batch)):
        if n % 1000 == 0:
            print("{}/{}".format(n, total_num))

        latent_vec = latents_raw[n * batch: (n + 1) * batch, :]
        label_vec = labels[n * batch: (n + 1) * batch, :]
        used_labels.append(label_vec)
        images = Gs.run(latent_vec, label_vec)
        images = np.clip(np.rint((images + 1.0) / 2.0 * 255.0), 0.0, 255.0).astype(np.uint8)  # [-1,1] => [0,255]
        images = images.transpose(0, 2, 3, 1)  # NCHW => NHWC
        save_images = np.squeeze(images, axis=-1)

        data_dir = output_data_path
        if not os.path.exists(data_dir):
            os.makedirs(data_dir)

        for idx in range(save_images.shape[0]):
            image_idx = idx + batch * n
            labels_save["{}_{}".format(cat, image_idx)] = labels[image_idx, :]
            store_name = 'fake_{}_{}.png'.format(cat, image_idx)
            used_imgname.append(store_name)
            store_path = os.path.join(data_dir, store_name)
            imageio.imwrite(store_path, save_images[idx])

    print("Done with {}".format(cat))
    print(len(labels))
    print(len(used_labels))
    print(len(used_imgname))
    sys.stdout.flush()

with open(output_data_path+ "gan_image_labels.pkl", "wb") as handle:
    pickle.dump(labels_save, handle)
sys.stdout.flush()

print("Done :)")

Lung Lesion: (15907, 14)
Pleural Other: (20228, 14)
Fracture: (16984, 14)
Lung Lesion: (15907, 14)
Pleural Other: (20228, 14)
Fracture: (16984, 14)
Generating Lung Lesion
0/15907
1000/15907
2000/15907
3000/15907
4000/15907
5000/15907
6000/15907
7000/15907
8000/15907
9000/15907
10000/15907
11000/15907
12000/15907
13000/15907
14000/15907
15000/15907
Done with Lung Lesion
15907
15907
15907
Generating Pleural Other
0/20228
1000/20228
2000/20228
3000/20228
4000/20228
5000/20228
6000/20228
7000/20228
8000/20228
9000/20228
10000/20228
11000/20228
12000/20228
13000/20228
14000/20228
15000/20228
16000/20228
17000/20228
18000/20228
19000/20228
20000/20228
Done with Pleural Other
20228
20228
20228
Generating Fracture
0/16984
1000/16984
2000/16984
3000/16984
4000/16984
5000/16984
6000/16984
7000/16984
8000/16984
9000/16984
10000/16984
11000/16984
12000/16984
13000/16984
14000/16984
15000/16984
16000/16984
Done with Fracture
16984
16984
16984
Done :)
